In [1]:
import numpy as np
import pandas as pd
import math
import itertools
import datetime
import csv

In [2]:
def countPath(path, arr, rozmiar):
    new_path=0
    new_path+=arr[0][path[0]]
    try:
        for step in range(rozmiar-2):
            new_path+=arr[path[step]][path[step+1]]
    except:
        print('new_path+=arr[path[step]][path[step+1]]')
        print("new_path:",new_path)
        print('path',path)
        print('path.size',path.size)
        print('step',step)
        print('rozmiar-2', rozmiar-2)
        print('arr.shape',arr.shape)
    new_path+=arr[path[rozmiar-2]][0]
    return new_path

In [3]:
def wczytanieWynikow():
    the_best_solutions = pd.read_csv('Dane\\wyniki.csv', header=None, sep=';', index_col=0, names=['costam','best_length'])
    return the_best_solutions

In [4]:
def wczytanieMacierzy(rozmiar):
    file = 'Dane/data{}.txt'.format(rozmiar)
    df = pd.read_csv(file, nrows=2, header=None)
    name = df[df.index==0].values[0][0]
    n = int(df[df.index==1].values[0][0])

    df = pd.read_csv(file, skiprows=2, sep='\s+', header=None)
    arr = df.values
    return arr

In [5]:
def swap_sciezka(zaburzona_sciezka,a,b):
    zaburzona_sciezka[a], zaburzona_sciezka[b] = zaburzona_sciezka[b], zaburzona_sciezka[a]
    return zaburzona_sciezka

In [6]:
def zaburzonePozycje(n):
    a,b = 0,0
    while a == b:
        a,b = np.random.randint(1, high=n-1, size=2)
    return a,b

In [7]:
def insert_sciezka(zaburzona_sciezka,a,b,n):
    while True:
        if b+1==a:
                a,b = zaburzonePozycje(n)
        else:
            break
    zaburzona_sciezka = np.insert(zaburzona_sciezka, a, zaburzona_sciezka[b])
    if a<b:
        zaburzona_sciezka = np.hstack((zaburzona_sciezka[:b+1],zaburzona_sciezka[b+2:]))
    else:
        zaburzona_sciezka = np.hstack((zaburzona_sciezka[:b],zaburzona_sciezka[b+1:]))
        
    return zaburzona_sciezka

In [8]:
def invert_sciezka(zaburzona_sciezka,a,b):
    if b<a:
        a, b = b, a
    while a<b:
        zaburzona_sciezka[a], zaburzona_sciezka[b] = zaburzona_sciezka[b], zaburzona_sciezka[a]
        a += 1
        b -= 1
        
    return zaburzona_sciezka

In [43]:
def najlepsze_z(populacja_poczatkowa):
    best = 9999999
    best_path = []
    for populacja in populacja_poczatkowa:
        if populacja[1]<best:
            best = populacja[1]
            best_path = populacja[2]
    return best, best_path

In [44]:
def normalizacja_r(prawdopodobienstwo):
    nowe_prawdopodobienstwo = []
    suma = 0
    for prawd in prawdopodobienstwo:
        suma += prawd
    for prawd in prawdopodobienstwo:
        nowe_prawdopodobienstwo.append(prawd/suma)
    return nowe_prawdopodobienstwo

In [45]:
def selekcja_calosciowa(populacja, wybor_selekcji=3, precyzja_ruletki=2, rozmiar_turnieju=2, metoda_selekcji_rankingowej=1,uwzglednienie_plci=False,wielkosc_populacji=100): #wybor i zmiana wszystkich par          
    populacja_arr = np.array(populacja)
    ilosc_par = int(len(populacja)/2)
    pary = []
    if wybor_selekcji==0: #Losowa:
        if uwzglednienie_plci:
            a = np.random.randint(0, int(wielkosc_populacji/2), size=[ilosc_par,1])
            b = np.random.randint(int(wielkosc_populacji/2), wielkosc_populacji, size=[ilosc_par,1])
            pary = np.append(a,b, axis=1)
        else:
            pary = np.random.randint(0, wielkosc_populacji, size=[ilosc_par,2])
    
    elif wybor_selekcji==1: #ruletka
        nr_pary = 0
        while nr_pary < ilosc_par:
            
            if precyzja_ruletki == 0: #bez zwiekszenia precji selekcyjnej  
                f_celu = np.array(populacja)
                f_celu = f_celu[:,0]
                rodzice = wybranie_rodzicow(f_celu)
                    
            elif precyzja_ruletki == 1: #zwiekszenie presji selekcyjnej (zwiekszenie preferencji wybierania lepszych rozwiazan)
                przerobione_f_celu = []
                stare_f_celu = np.array(populacja)
                stare_f_celu = stare_f_celu[:,0]
                max_ze_starych = np.max(stare_f_celu)
                for stara_f_celu in stare_f_celu:
                    nowa_f_celu = stara_f_celu/max_ze_starych
                    przerobione_f_celu.append(nowa_f_celu)
                rodzice = wybranie_rodzicow(przerobione_f_celu)

            elif precyzja_ruletki == 2: #wieksze zwiekszenie presji selekcyjnej (może być za duze)
                przerobione_f_celu = []
                stare_dlugosci = np.array(populacja)
                stare_dlugosci = stare_dlugosci[:,1]
                min_z_dlugosci = np.min(stare_dlugosci)
                for stara_dlugosc in stare_dlugosci:
                    nowa_dlugosc = stara_dlugosc - min_z_dlugosci + 1
                    przerobione_f_celu.append(1/nowa_dlugosc)
                rodzice = wybranie_rodzicow(przerobione_f_celu)
            
            pary.append(rodzice)        
            nr_pary += 1
        
    elif wybor_selekcji==2: #turniejowa
        nr_pary = 0
        while nr_pary < ilosc_par:
            rodzice = [0,0]
            for i in range(2):
                if uwzglednienie_plci:
                    uczestnicy_turnieju = np.random.randint(int(wielkosc_populacji/2)*i, int(wielkosc_populacji/(2-i)), size=[1,rozmiar_turnieju])
                else:
                    uczestnicy_turnieju = np.random.randint(0, wielkosc_populacji, size=[1,rozmiar_turnieju])
                rozmiary = []
                for j in range(rozmiar_turnieju):
                    rozmiary.append(populacja_arr[uczestnicy_turnieju[0,j],1])
                min_rozmiar = np.min(rozmiary)
                wygrany_rodzic = 0
                for j in range(rozmiar_turnieju):
                    if rozmiary[j] == min_rozmiar:
                        wygrany_rodzic = uczestnicy_turnieju[0, j]
                        break
                rodzice[i] = wygrany_rodzic
            
            pary.append(rodzice)        
            nr_pary += 1
            
    elif wybor_selekcji==3: #rankingowa
        prawdopodobienstwa = []
        rangi = np.array([], dtype=int)
        arr_argsort = np.argsort(populacja_arr[:,1]) 
        for i in range(len(populacja)):
            rangi = np.insert(rangi,0,i+1)
        if metoda_selekcji_rankingowej == 0: # liniowa
            for i in range(len(populacja)):
                wspolczynnik_selekcji = 2.0 #[1.0, 2.0]
                funkcja_R = 2-wspolczynnik_selekcji+(2*(wspolczynnik_selekcji-1)*(rangi[i]-1))/(len(populacja)-1)
                prawdopodobienstwa.append(funkcja_R)
        elif metoda_selekcji_rankingowej == 1: # nieliniowa
            wspolczynnik_selekcji = 1.41 #dobrany eksperymentalnie
            sum_wsp_selekcji = suma_wsp_selekcji(wspolczynnik_selekcji, len(populacja))
            for i in range(len(populacja)):
                funkcja_R = (len(populacja)*np.power(wspolczynnik_selekcji, rangi[i]-1)/(sum_wsp_selekcji))
                prawdopodobienstwa.append(funkcja_R)
                
        #Normalizacja funkcji R:
        prawdopodobienstwa = normalizacja_r(prawdopodobienstwa)
        nr_pary = 0
        while nr_pary < ilosc_par: #Teraz podobnie jak w ruletce
        
            losowe_liczby = np.random.rand(2)
            rodzice = [0, 0]
            for nr_liczby, losowa_liczba in enumerate(losowe_liczby): 
                suma_prawd = 0
                if uwzglednienie_plci:
                    prawdopodobienstwa = np.array(prawdopodobienstwa)*2
                    if nr_liczby==1:
                        losowa_liczba += 1
                for num, prawdopodobienstwo in enumerate(prawdopodobienstwa):
                    suma_prawd += prawdopodobienstwo
                    if (num == len(prawdopodobienstwa)-1) or (losowa_liczba < suma_prawd):
                        rodzice[nr_liczby] = arr_argsort[num]
                        break
            pary.append(rodzice)        
            nr_pary += 1
        
    return pary

In [46]:
def suma_wsp_selekcji(wspolczynnik_selekcji, dlugosc):
    suma = 0
    for i in range(dlugosc):
        suma += np.power(wspolczynnik_selekcji,i)
    return suma

In [47]:
def wybranie_rodzicow(f_celu, uwzglednienie_plci=False): #plec sprawia, ze rodzica z pierwszej polowy poopulacji
    suma_f_celu = np.sum(f_celu)                         #krzyzujemy z rodzicem z drugiej polowy
    losowe_liczby = np.random.rand(2)
    rodzice = [0, 0]
    for nr_liczby, losowa_liczba in enumerate(losowe_liczby):           
        prawdopodobienstwa = []
        for cel in f_celu:
            prawdopodobienstwa.append(cel/suma_f_celu)
        suma_prawd = 0
        
        if uwzglednienie_plci:
            prawdopodobienstwa = np.array(prawdopodobienstwa)*2
            if nr_liczby==1:
                losowa_liczba += 1
            
        for num, prawdopodobienstwo in enumerate(prawdopodobienstwa):
            suma_prawd += prawdopodobienstwo
            if (num == len(prawdopodobienstwa)-1) or (losowa_liczba < suma_prawd):
                rodzice[nr_liczby] = num
                break
                
    return rodzice

In [48]:
def rekurencyjne_szukanie(rodzic1, rodzic2, item, przedzial1, przedzial2):    
    index = np.argwhere(np.array(rodzic2)==item)[0,0]
    item2 = rodzic1[index]
    if item2 in przedzial2:
        miejsce = rekurencyjne_szukanie(rodzic1, rodzic2, item2, przedzial1, przedzial2)
    else:
        miejsce = np.argwhere(np.array(rodzic2)==item2)[0,0]
    return miejsce

In [49]:
def daj_dziecko(dziecko1, dziecko2, rodzic1, rodzic2, przedzial1, przedzial2):
    for item in przedzial2:
        if item not in przedzial1:
            miejsce = rekurencyjne_szukanie(rodzic1, rodzic2, item, przedzial1, przedzial2)
            dziecko1[miejsce] = item
    
    for i in range(dziecko1.size):
        if dziecko1[i]==0:
            dziecko1[i] = rodzic2[i]
    return dziecko1

In [50]:
def krzyzowanie(para, operator=2):
    rodzic1 = para[0]
    rodzic2 = para[1]
    
    if operator == 0: # jednopunktowa - pierwsza polowa taka sama, druga to dopisanie liczb z drugiego rodzica
        miejsce_podzialu=int(len(rodzic1)/2)
        rodzic1_half = rodzic1[:miejsce_podzialu]
        rodzic1_copy = rodzic1.copy()
        rodzic2_half = rodzic2[:miejsce_podzialu]
        rodzic2_copy = rodzic2.copy()

        for item in rodzic2_half:
            rodzic1_copy = rodzic1_copy[rodzic1_copy!=item]
        for item in rodzic1_half:
            rodzic2_copy = rodzic2_copy[rodzic2_copy!=item]

        dziecko1 = np.array(list(rodzic2_half) + list(rodzic1_copy))
        dziecko2 = np.array(list(rodzic1_half) + list(rodzic2_copy))
        
    if operator == 1: #OX
        k1, k2 = np.random.randint(0, len(rodzic1), size=[1,2])[0]
        if k1>k2:
            k1,k2=k2,k1
        
        dziecko1 = rodzic1[k1:k2+1]
        dziecko2 = rodzic2[k1:k2+1]
        n_r_2 = np.array(list(rodzic2[k2+1:])+list(rodzic2[:k2+1]))
        n_r_1 = np.array(list(rodzic1[k2+1:])+list(rodzic1[:k2+1]))

        for value in n_r_2:
            if value not in dziecko1:
                dziecko1 = np.append(dziecko1, value )
        dziecko1 = np.append(dziecko1[len(dziecko1)-k1:], dziecko1[:len(dziecko1)-k1])

        for value in n_r_1:
            if value not in dziecko2:
                dziecko2 = np.append(dziecko2, value )
        dziecko2 = np.append(dziecko2[len(dziecko2)-k1:], dziecko2[:len(dziecko2)-k1])

    if operator == 2: #PMX
        k1, k2 = np.random.randint(0, len(rodzic1), size=[1,2])[0]
        if k1>k2:
            k1,k2=k2,k1
            
        dziecko1 = np.zeros(len(rodzic1))
        dziecko2 = np.zeros(len(rodzic1))
        
        przedzial1=rodzic1[k1:k2+1]
        przedzial2=rodzic2[k1:k2+1]
        
        dziecko1[k1:k2+1] = przedzial1
        dziecko2[k1:k2+1] = rodzic2[k1:k2+1]
        
        dziecko1 = daj_dziecko(dziecko1, dziecko2, rodzic1,rodzic2, przedzial1, przedzial2)
        dziecko2 = daj_dziecko(dziecko2, dziecko1, rodzic2,rodzic1, przedzial2, przedzial1)

    return dziecko1, dziecko2

In [51]:
def mutacja(path, wariant=2, rozmiar=10):
    a,b = zaburzonePozycje(rozmiar)
    
    if wariant==0: #SWAP
        path = swap_sciezka(path,a,b)
    
    if wariant==1:
        path = insert_sciezka(path,a,b,rozmiar)
        
    if wariant==2:
        path = invert_sciezka(path,a,b)
        
    return path

In [69]:
def ls_greedy(arr,populacje, jedna_iteracja=True, rozmiar=10):
    nowe_populacje = populacje.copy()
    for nr, populacja in enumerate(populacje):
        path = populacja[2]
        length = populacja[1]
        while True:
            best_new_path = []
            best_new_length = length
            nowy=False
            for i in range(len(path)-1):
                for j in range(i+1, len(path)):
                    new_path = path.copy()
                    new_path[i],new_path[j]=new_path[j],new_path[i]
                    new_length = new_length = countPathFaster(length, path, new_path, arr, rozmiar,i,j)
                    #new_length = countPath(new_path, arr, rozmiar)
                    if new_length < best_new_length:
                        best_new_length = new_length
                        best_new_path = new_path
                        nowy=True
                        break
                if nowy:
                    break
            if best_new_length < length:
                length = best_new_length
                path = best_new_path
                if jedna_iteracja:
                    break
            else:
                break
            
        nowe_populacje[nr][2] = path
        nowe_populacje[nr][1] = length
        nowe_populacje[nr][0] = 1/length
    return nowe_populacje

In [70]:
def countPathFaster(old_length, old_path, path, arr, rozmiar,i,j):
    length = old_length
    if i==0:
        length -= arr[0][old_path[0]]
        length -= arr[old_path[0]][old_path[1]]
        
        length += arr[0][path[0]]
        length += arr[path[0]][path[1]]
    else:
        length -= arr[old_path[i-1]][old_path[i]]
        length -= arr[old_path[i]][old_path[i+1]]
        
        length += arr[path[i-1]][path[i]]
        length += arr[path[i]][path[i+1]]
        
    if j== rozmiar-2:
        length -= arr[old_path[j-1]][old_path[j]]
        length -= arr[old_path[j]][old_path[0]]
        
        length += arr[path[j-1]][path[j]]
        length += arr[path[j]][path[0]]
    else:
        length -= arr[old_path[j-1]][old_path[j]]
        length -= arr[old_path[j]][old_path[j+1]]
        
        length += arr[path[j-1]][path[j]]
        length += arr[path[j]][path[j+1]]
    
    
    return length

In [71]:
def ls_steepest(arr,populacje, jedna_iteracja=True, rozmiar=10):
    nowe_populacje = populacje.copy()
    for nr, populacja in enumerate(populacje):
        path = populacja[2]
        length = populacja[1]
        while True:
            best_new_path = []
            best_new_length = length
            for i in range(len(path)-1):
                for j in range(i+1, len(path)):
                    new_path = path.copy()
                    new_path[i],new_path[j]=new_path[j],new_path[i]
                    new_length = countPathFaster(length, path, new_path, arr, rozmiar,i,j)
                    #new_length = countPath(new_path, arr, rozmiar)
                    if new_length < best_new_length:
                        best_new_length = new_length
                        best_new_path = new_path

            if best_new_length < length:
                length = best_new_length
                path = best_new_path
                if jedna_iteracja:
                    break
            else:
                break
            
        nowe_populacje[nr][2] = path
        nowe_populacje[nr][1] = length
        nowe_populacje[nr][0] = 1/length
    return nowe_populacje

In [72]:
def ls_2opt(arr,populacja, wersja_ls, jedna_iteracja, rozmiar):
    #nowe_populacje = populacja.copy()
    if wersja_ls==0:
        nowe_populacje = ls_greedy(arr,populacja, jedna_iteracja=jedna_iteracja, rozmiar=rozmiar) #first improvement, first descent
    if wersja_ls==1:
        nowe_populacje = ls_steepest(arr,populacja, jedna_iteracja=jedna_iteracja, rozmiar=rozmiar) #best impovement, highest descent
    
    return nowe_populacje

In [73]:
# rozmiar = 42
# nazwa = 'data{}'.format(rozmiar)
# arr = wczytanieMacierzy(rozmiar)
# the_best_solutions = wczytanieWynikow()
# best_solution = the_best_solutions.loc[[nazwa],['best_length']].values[0,0]

In [74]:
# warianty_mutacji = [2]
# czy_elitarny = [True]
# wielkosci_populacji = [100]
# prawdopodobienstwa_krzyzowania = [0.9]
# prawdopodobienstwa_mutacji = [0.1]
# elitaryzm=True
# # Paramtery do Local Search
# czy_hybrydowy = True # czy uzywamy Local Search dla populacji
# wersja_ls = 0 # wersja Local Search: 0-greedy, 1-steepest
# czy_jedna_iteracja = True # ile iteracji w Local Search 

# #Parametry do Selekcji
# nr_selekcji = 3 # 0-losowa, 1-ruletka, 2-turniejowa, 3-rankingowa
# precyzja_ruletki = 2 # 0-bez, 1-male, 2-duze
# ilosc_uczestnikow_turnieju = 5 # ile populacji staje naraz do turnieju
# metoda_selekcji_rankingowej = 1 # 0-liniowa, 1-nieliniowa
# czy_rozroznienie_plci = False # jezeli True, to pierwsza polowa populacji jest jednej plci niz druga, krzyzowac moga sie tylko populacje z roznych polow

# #Parametry Krzyzowania
# metoda_krzyzowania = 2 # 0 - jednopunktowe, 1- OX, 2- PMX

# #Parametry Mutowania
# wariant_mutowania = 2 # 0 - swap, 1- insert, 2 - invert

# wielkosc_populacji = 100
# nr_pokolenia = 1
# p_k = 0.9 # Prawdopodobienstwo krzyzowania, 0.6-1.0
# p_m = 0.1 # Prawdopodobienstwo mutacji, 0.01 - 0.1
# max_czas = 7.0 # maksymalny czas dzialania algorytmu

# struct = {
#             'rozmiar' : 0, #3
#             'czy_hybrydowy' : True, #3*2=6
#             'wersja_ls' : 0, #3*2=6
#             'czy_jedna_iteracja_w_ls' : -1, #6+1*3=9
#             'czy_rozroznienie_plci' : True, #9*2=18
#             'nr_selekcji' : 0, #18*4=72
#             'precyzja_ruletki' : -1, #72+2*18=108
#             'ilosc_uczestnikow_turnieju' : -1, #108+1*18=126
#             'metoda_selekcji_rankingowej' : -1, #126+1*18=144
#             'metoda_krzyzowania' : 0, #144*3=432
#             'jakosc_wyniku' : 0 #432*10
#         }

In [75]:
def obliczenieCzasuWyniku(arr, rozmiar,czy_hybrydowy,wersja_ls,czy_jedna_iteracja_w_ls,czy_rozroznienie_plci,nr_selekcji,precyzja_ruletki,ilosc_uczestnikow_turnieju,metoda_selekcji_rankingowej,metoda_krzyzowania):
    elitaryzm=True
    wielkosc_populacji = 100
    nr_pokolenia = 1
    p_k = 0.9 # Prawdopodobienstwo krzyzowania, 0.6-1.0
    p_m = 0.1 # Prawdopodobienstwo mutacji, 0.01 - 0.1
    max_czas = 7.0 # maksymalny czas dzialania algorytmu
    wariant_mutowania = 2
    populacja_poczatkowa = [] #funckja celu = 1/dlugosc, dlugosc, sciezka
    for i in range(wielkosc_populacji):
        path = np.random.permutation(range(1,rozmiar))
        length = countPath(path, arr, rozmiar)
        populacja_poczatkowa.append([1/length,length,path])

    najkrotsza_dlugosc, najlepsze_rozwiazanie = najlepsze_z(populacja_poczatkowa)


    start = datetime.datetime.now()
    duration = datetime.datetime.now() - start
    populacja = populacja_poczatkowa

    arr_pop=np.array(populacja)
    elitarni_osobnicy=[]
    if elitaryzm:
        elitarny = populacja_poczatkowa[np.argmin(arr_pop[:,1], axis=0)]

    while duration.total_seconds() < max_czas:
        # wybranie wielkosc_populacji/2 par
        pary = selekcja_calosciowa(populacja, wybor_selekcji=nr_selekcji, precyzja_ruletki=precyzja_ruletki, rozmiar_turnieju=ilosc_uczestnikow_turnieju, metoda_selekcji_rankingowej=1,uwzglednienie_plci=czy_rozroznienie_plci,wielkosc_populacji=wielkosc_populacji)
        j = 0
        k = int(len(populacja_poczatkowa)/2)
        populacja_nowa = []
        while j<k:
            dziecko1 = populacja[pary[j][0]][2]
            dziecko2 = populacja[pary[j][1]][2]
            dzieci = []
            dzieci.append(dziecko1)
            dzieci.append(dziecko2)
            R = np.random.rand(3)
            if R[0] < p_k:
                dziecko1, dziecko2 = krzyzowanie(dzieci, operator=metoda_krzyzowania)
            if R[1] < p_m:
                dziecko1 = mutacja(dziecko1, wariant=wariant_mutowania, rozmiar=rozmiar)
            if R[2] < p_m:
                dziecko2 = mutacja(dziecko2, wariant=wariant_mutowania, rozmiar=rozmiar)

            dziecko1 = dziecko1.astype(int)
            dziecko2 = dziecko2.astype(int)
            length1 = countPath(dziecko1, arr, rozmiar)
            length2 = countPath(dziecko2, arr, rozmiar)
            populacja_nowa.append([1/length1,length1,dziecko1])
            populacja_nowa.append([1/length2,length2,dziecko2])

            j += 1 

#         nr_pokolenia += 1
        populacja = populacja_nowa
        najkrotsza_dlugosc_z_obecnej_populacji, najlepsze_rozwiazanie_z_obecnej_populacji = najlepsze_z(populacja)
        if najkrotsza_dlugosc_z_obecnej_populacji < najkrotsza_dlugosc:
            najkrotsza_dlugosc = najkrotsza_dlugosc_z_obecnej_populacji
            najlepsze_rozwiazanie = najlepsze_rozwiazanie_z_obecnej_populacji

        arr_pop=np.array(populacja)
        if elitaryzm:
            najgorszy = np.argmax(arr_pop[:,1], axis=0)
            if populacja[najgorszy][1] > elitarny[1]:
                populacja[najgorszy] = elitarny

            arr_pop=np.array(populacja)   
            elitarny = populacja[np.argmin(arr_pop[:,1], axis=0)]

        if czy_hybrydowy:
            populacja = ls_2opt(arr,populacja, wersja_ls=wersja_ls, jedna_iteracja=czy_jedna_iteracja_w_ls, rozmiar=rozmiar)

        duration = datetime.datetime.now() - start

    return najkrotsza_dlugosc
# print('uzyskana dlugosc -> najlepsza dlugosc')
# print(najkrotsza_dlugosc, "->", best_solution)
# #print('path->',najlepsze_rozwiazanie)
# print('nr_pokolenia ->',nr_pokolenia)

In [76]:
# def obliczenia(rozmiary, liczba_powtorzen, csvwriter, struct):
#     rozmiary = [21, 71, 323]
#     liczba_powtorzen = [10, 10, 10]
#     czy_hybrydowy_lista = [True, False]
#     wersje_ls = [0,1]
#     czy_jedna_iteracja_w_ls_lista = [True, False]
# #    czy_rozroznienie_plci_lista = [True, False]
# #     selekcje = [0,1,2,3]
# #     precyzje_ruletki = [0,1,2]
# #     ilosci_uczestnikow_turnieju = [2, 10]
# #     metody_selekcji_rankingowej = [0,1]
#     selekcje = [0,1,2,3,4,5,6,7] #0-zwykla, 1,2,3-precyzje_ruletki, 4,5-ilosci_ucz_turn, 6,7 - rankingowa
#     metody_krzyzowania = [0,1,2]
    
#     jakosc_wynikow = []    
#     the_best_solutions = wczytanieWynikow()
    
#     for nr, rozmiar in enumerate(rozmiary): #3 rozmiarow
#         struct['rozmiar'] = rozmiar
#         ilosc_powtorzen = liczba_powtorzen[nr]
#         jakosc_wyniku = 0.0
#         nazwa = 'data{}'.format(rozmiar)
#         print('\nRozmiar:',rozmiar, end=',')
#         jakosc_hybrydy = []
#         for czy_hybrydowy in czy_hybrydowy_lista:  #3*2=6  
#             print(' | czy_hybrydowy:',czy_hybrydowy, end=',')
#             struct['czy_hybrydowy'] = czy_hybrydowy
#             for wersja_ls in wersje_ls: #6*2=12
#                 print(' wersja_ls:',wersja_ls, end=',')
#                 struct['wersja_ls'] = wersja_ls
#                 for czy_jedna_iteracja_w_ls in czy_jedna_iteracja_w_ls_lista: #12*2=24
#                     struct['czy_jedna_iteracja_w_ls'] = czy_jedna_iteracja_w_ls
#                     for czy_rozroznienie_plci in czy_rozroznienie_plci_lista: #24*2=48
#                         struct['czy_rozroznienie_plci'] = czy_rozroznienie_plci
#                         for nr_selekcji in selekcje: #48*4=192
#                             struct['nr_selekcji'] = nr_selekcji
#                             for precyzja_ruletki in precyzje_ruletki: #192*3=576
#                                 struct['precyzja_ruletki'] = precyzja_ruletki
#                                 for ilosc_uczestnikow_turnieju in ilosci_uczestnikow_turnieju: #576*2=1152
#                                     struct['ilosc_uczestnikow_turnieju'] = ilosc_uczestnikow_turnieju
#                                     for metoda_selekcji_rankingowej in metody_selekcji_rankingowej: #1152*2=2304
#                                         struct['metoda_selekcji_rankingowej'] = metoda_selekcji_rankingowej
#                                         for metoda_krzyzowania in metody_krzyzowania: #2304*3=6912
#                                             struct['metoda_krzyzowania'] = metoda_krzyzowania
#                                             jakosc_wyniku = 0.0
#                                             for powtorzenie in range(ilosc_powtorzen): #6912*10=69120
#                                                 arr = wczytanieMacierzy(rozmiar)
#                                                 #69120*7s=
#                                                 wynik = obliczenieCzasuWyniku(arr, rozmiar,czy_hybrydowy,wersja_ls,czy_jedna_iteracja_w_ls,czy_rozroznienie_plci,nr_selekcji,precyzja_ruletki,ilosc_uczestnikow_turnieju,metoda_selekcji_rankingowej,metoda_krzyzowania)
#                                                 nowa_jakosc_wyniku = the_best_solutions.loc[[nazwa],['best_length']].values[0,0]/wynik

#                                                 if nowa_jakosc_wyniku > jakosc_wyniku:
#                                                     jakosc_wyniku = nowa_jakosc_wyniku

#                                             struct['jakosc_wyniku'] = jakosc_wyniku
#                                             csvwriter.writerow(struct)
                        
                        
    
#     return jakosc_wynikow

In [85]:
def obliczenia(rozmiary, liczba_powtorzen, csvwriter, struct):
    rozmiary = [21, 71, 323]
    liczba_powtorzen = [10, 10, 10]
    czy_hybrydowy_lista = [0,1,2]
#     wersje_ls = [0,1]
#     czy_jedna_iteracja_w_ls_lista = [True]
#    czy_rozroznienie_plci_lista = [True, False]
#     selekcje = [0,1,2,3]
#     precyzje_ruletki = [0,1,2]
#     ilosci_uczestnikow_turnieju = [2, 10]
#     metody_selekcji_rankingowej = [0,1]
    selekcje = [0,1,2,3,4,5,6,7] #0-zwykla, 1,2,3-precyzje_ruletki, 4,5-ilosci_ucz_turn, 6,7 - rankingowa
    metody_krzyzowania = [0,1,2]
    
    jakosc_wynikow = []    
    the_best_solutions = wczytanieWynikow()
    
    start = datetime.datetime.now()
    czas_caly = datetime.datetime.now() - start
    czas_pojedynczy = datetime.datetime.now() - start
    nr_wyniku = 1
    
    for nr, rozmiar in enumerate(rozmiary): #5 rozmiarow
        struct['rozmiar'] = rozmiar
        ilosc_powtorzen = liczba_powtorzen[nr]
        jakosc_wyniku = 0.0
        nazwa = 'data{}'.format(rozmiar)
#         print('\nRozmiar:',rozmiar, end=',')
        jakosc_hybrydy = []
        for czy_hybrydowy_duzy in czy_hybrydowy_lista:  #5*3=15  
            if czy_hybrydowy_duzy == 0:
                struct['czy_hybrydowy'] = True
                struct['wersja_ls'] = 0
                struct['czy_jedna_iteracja_w_ls']=True
            elif czy_hybrydowy_duzy == 1:
                struct['czy_hybrydowy'] = True
                struct['wersja_ls'] = 1
                struct['czy_jedna_iteracja_w_ls']=True
            elif czy_hybrydowy_duzy == 2:
                struct['czy_hybrydowy'] = False
                struct['wersja_ls'] = -1
                struct['czy_jedna_iteracja_w_ls']=-1
                
            for nr_selekcji_duzy in selekcje: #15*8=120
                if nr_selekcji_duzy==0:
                    struct['nr_selekcji'] = 0
                    struct['precyzja_ruletki'] = -1
                    struct['ilosc_uczestnikow_turnieju'] = -1
                    struct['metoda_selekcji_rankingowej'] = -1
                elif nr_selekcji_duzy==1:
                    struct['nr_selekcji'] = 1
                    struct['precyzja_ruletki'] = 0
                    struct['ilosc_uczestnikow_turnieju'] = -1
                    struct['metoda_selekcji_rankingowej'] = -1
                elif nr_selekcji_duzy==2:
                    struct['nr_selekcji'] = 1
                    struct['precyzja_ruletki'] = 1
                    struct['ilosc_uczestnikow_turnieju'] = -1
                    struct['metoda_selekcji_rankingowej'] = -1
                elif nr_selekcji_duzy==3:
                    struct['nr_selekcji'] = 1
                    struct['precyzja_ruletki'] = 2
                    struct['ilosc_uczestnikow_turnieju'] = -1
                    struct['metoda_selekcji_rankingowej'] = -1
                elif nr_selekcji_duzy==4:
                    struct['nr_selekcji'] = 2
                    struct['precyzja_ruletki'] = -1
                    struct['ilosc_uczestnikow_turnieju'] = 2
                    struct['metoda_selekcji_rankingowej'] = -1
                elif nr_selekcji_duzy==5:
                    struct['nr_selekcji'] = 2
                    struct['precyzja_ruletki'] = -1
                    struct['ilosc_uczestnikow_turnieju'] = 10
                    struct['metoda_selekcji_rankingowej'] = -1
                elif nr_selekcji_duzy==6:
                    struct['nr_selekcji'] = 3
                    struct['precyzja_ruletki'] = -1
                    struct['ilosc_uczestnikow_turnieju'] = -1
                    struct['metoda_selekcji_rankingowej'] = 0
                elif nr_selekcji_duzy==7:
                    struct['nr_selekcji'] = 3
                    struct['precyzja_ruletki'] = -1
                    struct['ilosc_uczestnikow_turnieju'] = -1
                    struct['metoda_selekcji_rankingowej'] = 1

                for metoda_krzyzowania in metody_krzyzowania: #120*3=360
                    struct['metoda_krzyzowania'] = metoda_krzyzowania
                    jakosc_wyniku = 0.0
                    for powtorzenie in range(ilosc_powtorzen): #360*10=3600
                        arr = wczytanieMacierzy(rozmiar)
                        #3600*7s=
                        wynik = obliczenieCzasuWyniku(arr, rozmiar,struct['czy_hybrydowy'],struct['wersja_ls'],struct['czy_jedna_iteracja_w_ls'],False,struct['nr_selekcji'],struct['precyzja_ruletki'],struct['ilosc_uczestnikow_turnieju'],struct['metoda_selekcji_rankingowej'],metoda_krzyzowania)
                        nowa_jakosc_wyniku = the_best_solutions.loc[[nazwa],['best_length']].values[0,0]/wynik

                        if nowa_jakosc_wyniku > jakosc_wyniku:
                            jakosc_wyniku = nowa_jakosc_wyniku

                    struct['jakosc_wyniku'] = jakosc_wyniku
                    csvwriter.writerow(struct)
                    czas_pojedynczy = datetime.datetime.now() - start - czas_caly
                    czas_caly = datetime.datetime.now() - start
                    print(nr_wyniku,czas_pojedynczy,czas_caly)
                    print(struct)
                    nr_wyniku+=1
                        
    
    return jakosc_wynikow

In [86]:
struct = {
            'rozmiar' : 0, #5
            'czy_hybrydowy' : True, #5*2=10
            'wersja_ls' : 0, #10*2=20
            'czy_jedna_iteracja_w_ls' : -1, #6+1*3=9
            'nr_selekcji' : 0, #18*4=72
            'precyzja_ruletki' : -1, #72+2*18=108
            'ilosc_uczestnikow_turnieju' : -1, #108+1*18=126
            'metoda_selekcji_rankingowej' : -1, #126+1*18=144
            'metoda_krzyzowania' : 0, #144*3=432
            'jakosc_wyniku' : 0 #432*10
        }

with open('algorytm_genetyczny.csv','w') as csvfile:
    fieldnames = ['rozmiar', 'czy_hybrydowy','wersja_ls','czy_jedna_iteracja_w_ls','czy_rozroznienie_plci','nr_selekcji','precyzja_ruletki','ilosc_uczestnikow_turnieju','metoda_selekcji_rankingowej','metoda_krzyzowania', 'jakosc_wyniku']
    csvwriter = csv.DictWriter(csvfile, fieldnames=fieldnames)
    csvwriter.writeheader()

    rozmiary = [21, 48, 71, 120, 323]
    liczba_powtorzen = [10, 10, 10, 10, 10]
    jakosc_wynikow = obliczenia(rozmiary, liczba_powtorzen, csvwriter, struct)

1 0:01:10.248665 0:01:10.248665
{'rozmiar': 21, 'czy_hybrydowy': True, 'wersja_ls': 0, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 0, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 0, 'jakosc_wyniku': 1.0}
2 0:01:10.270843 0:02:20.519508
{'rozmiar': 21, 'czy_hybrydowy': True, 'wersja_ls': 0, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 0, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 1, 'jakosc_wyniku': 1.0}
3 0:01:10.218744 0:03:30.738252
{'rozmiar': 21, 'czy_hybrydowy': True, 'wersja_ls': 0, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 0, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 2, 'jakosc_wyniku': 0.8840627041149576}
4 0:01:10.753626 0:04:41.491878
{'rozmiar': 21, 'czy_hybrydowy': True, 'wersja_ls': 0, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 1, 'precyzja_rul

C:\Users\kulaw\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in longlong_scalars


12 0:01:10.362059 0:14:05.095569
{'rozmiar': 21, 'czy_hybrydowy': True, 'wersja_ls': 0, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 1, 'precyzja_ruletki': 2, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 2, 'jakosc_wyniku': 1.0}
13 0:01:10.976456 0:15:16.072025
{'rozmiar': 21, 'czy_hybrydowy': True, 'wersja_ls': 0, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 2, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': 2, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 0, 'jakosc_wyniku': 1.0}
14 0:01:10.431736 0:16:26.503761
{'rozmiar': 21, 'czy_hybrydowy': True, 'wersja_ls': 0, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 2, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': 2, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 1, 'jakosc_wyniku': 1.0}
15 0:01:10.814053 0:17:37.317814
{'rozmiar': 21, 'czy_hybrydowy': True, 'wersja_ls': 0, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 2, 'precyzja_ruletki': -1, 'il

42 0:01:11.068971 0:49:32.009512
{'rozmiar': 21, 'czy_hybrydowy': True, 'wersja_ls': 1, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 2, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': 10, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 2, 'jakosc_wyniku': 1.0}
43 0:01:11.059511 0:50:43.069023
{'rozmiar': 21, 'czy_hybrydowy': True, 'wersja_ls': 1, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 3, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': 0, 'metoda_krzyzowania': 0, 'jakosc_wyniku': 1.0}
44 0:01:10.857514 0:51:53.926537
{'rozmiar': 21, 'czy_hybrydowy': True, 'wersja_ls': 1, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 3, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': 0, 'metoda_krzyzowania': 1, 'jakosc_wyniku': 1.0}
45 0:01:11.014976 0:53:04.941513
{'rozmiar': 21, 'czy_hybrydowy': True, 'wersja_ls': 1, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 3, 'precyzja_ruletki': -1, 'i

71 0:01:10.307756 1:23:32.253692
{'rozmiar': 21, 'czy_hybrydowy': False, 'wersja_ls': -1, 'czy_jedna_iteracja_w_ls': -1, 'nr_selekcji': 3, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': 1, 'metoda_krzyzowania': 1, 'jakosc_wyniku': 1.0}
72 0:01:10.194608 1:24:42.448300
{'rozmiar': 21, 'czy_hybrydowy': False, 'wersja_ls': -1, 'czy_jedna_iteracja_w_ls': -1, 'nr_selekcji': 3, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': 1, 'metoda_krzyzowania': 2, 'jakosc_wyniku': 0.9657509810916874}
73 0:01:10.485233 1:25:52.933533
{'rozmiar': 71, 'czy_hybrydowy': True, 'wersja_ls': 0, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 0, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 0, 'jakosc_wyniku': 0.668495029139527}
74 0:01:10.640215 1:27:03.573748
{'rozmiar': 71, 'czy_hybrydowy': True, 'wersja_ls': 0, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 

99 0:01:17.093327 1:57:05.364284
{'rozmiar': 71, 'czy_hybrydowy': True, 'wersja_ls': 1, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 0, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 2, 'jakosc_wyniku': 0.2808179723502304}
100 0:01:15.402567 1:58:20.766851
{'rozmiar': 71, 'czy_hybrydowy': True, 'wersja_ls': 1, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 1, 'precyzja_ruletki': 0, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 0, 'jakosc_wyniku': 0.2671598849157419}
101 0:01:15.439111 1:59:36.205962
{'rozmiar': 71, 'czy_hybrydowy': True, 'wersja_ls': 1, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 1, 'precyzja_ruletki': 0, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 1, 'jakosc_wyniku': 0.27981058975462764}
102 0:01:13.394234 2:00:49.600196
{'rozmiar': 71, 'czy_hybrydowy': True, 'wersja_ls': 1, 'czy_jedna_iteracja_w_ls': T

127 0:01:10.342122 2:32:48.388734
{'rozmiar': 71, 'czy_hybrydowy': False, 'wersja_ls': -1, 'czy_jedna_iteracja_w_ls': -1, 'nr_selekcji': 1, 'precyzja_ruletki': 1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 0, 'jakosc_wyniku': 0.30101883297313986}
128 0:01:10.606552 2:33:58.995286
{'rozmiar': 71, 'czy_hybrydowy': False, 'wersja_ls': -1, 'czy_jedna_iteracja_w_ls': -1, 'nr_selekcji': 1, 'precyzja_ruletki': 1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 1, 'jakosc_wyniku': 0.2790497996565541}
129 0:01:10.614111 2:35:09.609397
{'rozmiar': 71, 'czy_hybrydowy': False, 'wersja_ls': -1, 'czy_jedna_iteracja_w_ls': -1, 'nr_selekcji': 1, 'precyzja_ruletki': 1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 2, 'jakosc_wyniku': 0.28001148765077544}
130 0:01:10.334923 2:36:19.944320
{'rozmiar': 71, 'czy_hybrydowy': False, 'wersja_ls': -1, 'czy_jedna_iteracja_w_ls'

155 0:01:12.028395 3:05:53.935223
{'rozmiar': 323, 'czy_hybrydowy': True, 'wersja_ls': 0, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 1, 'precyzja_ruletki': 2, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 1, 'jakosc_wyniku': 0.2753322259136213}
156 0:01:11.581359 3:07:05.516582
{'rozmiar': 323, 'czy_hybrydowy': True, 'wersja_ls': 0, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 1, 'precyzja_ruletki': 2, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 2, 'jakosc_wyniku': 0.2787471095228085}
157 0:01:11.149168 3:08:16.665750
{'rozmiar': 323, 'czy_hybrydowy': True, 'wersja_ls': 0, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 2, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': 2, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 0, 'jakosc_wyniku': 0.2747047855811063}
158 0:01:12.341699 3:09:29.007449
{'rozmiar': 323, 'czy_hybrydowy': True, 'wersja_ls': 0, 'czy_jedna_iteracja_w_ls'

183 0:06:36.496338 5:03:32.645633
{'rozmiar': 323, 'czy_hybrydowy': True, 'wersja_ls': 1, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 2, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': 2, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 2, 'jakosc_wyniku': 0.23481494598902072}
184 0:06:34.910251 5:10:07.555884
{'rozmiar': 323, 'czy_hybrydowy': True, 'wersja_ls': 1, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 2, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': 10, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 0, 'jakosc_wyniku': 0.23304042179261863}
185 0:06:42.175672 5:16:49.731556
{'rozmiar': 323, 'czy_hybrydowy': True, 'wersja_ls': 1, 'czy_jedna_iteracja_w_ls': True, 'nr_selekcji': 2, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': 10, 'metoda_selekcji_rankingowej': -1, 'metoda_krzyzowania': 1, 'jakosc_wyniku': 0.23556581986143188}
186 0:06:40.367750 5:23:30.099306
{'rozmiar': 323, 'czy_hybrydowy': True, 'wersja_ls': 1, 'czy_jedna_iteracja_

211 0:01:11.058886 6:26:26.808409
{'rozmiar': 323, 'czy_hybrydowy': False, 'wersja_ls': -1, 'czy_jedna_iteracja_w_ls': -1, 'nr_selekcji': 3, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': 0, 'metoda_krzyzowania': 0, 'jakosc_wyniku': 0.2529568866844716}
212 0:01:12.052955 6:27:38.861364
{'rozmiar': 323, 'czy_hybrydowy': False, 'wersja_ls': -1, 'czy_jedna_iteracja_w_ls': -1, 'nr_selekcji': 3, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': 0, 'metoda_krzyzowania': 1, 'jakosc_wyniku': 0.28882596384230014}
213 0:01:11.226317 6:28:50.087681
{'rozmiar': 323, 'czy_hybrydowy': False, 'wersja_ls': -1, 'czy_jedna_iteracja_w_ls': -1, 'nr_selekcji': 3, 'precyzja_ruletki': -1, 'ilosc_uczestnikow_turnieju': -1, 'metoda_selekcji_rankingowej': 0, 'metoda_krzyzowania': 2, 'jakosc_wyniku': 0.26967663209273945}
214 0:01:10.942832 6:30:01.030513
{'rozmiar': 323, 'czy_hybrydowy': False, 'wersja_ls': -1, 'czy_jedna_iteracja_w

In [87]:
df = pd.read_csv('algorytm_genetyczny.csv')
df

,rozmiar,czy_hybrydowy,wersja_ls,czy_jedna_iteracja_w_ls,czy_rozroznienie_plci,nr_selekcji,precyzja_ruletki,ilosc_uczestnikow_turnieju,metoda_selekcji_rankingowej,metoda_krzyzowania,jakosc_wyniku
0,21,True,0,True,NaN,0,-1,-1,-1,0,1.000000
1,21,True,0,True,NaN,0,-1,-1,-1,1,1.000000
2,21,True,0,True,NaN,0,-1,-1,-1,2,0.884063
3,21,True,0,True,NaN,1,0,-1,-1,0,1.000000
4,21,True,0,True,NaN,1,0,-1,-1,1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
211,323,False,-1,-1,NaN,3,-1,-1,0,1,0.288826
212,323,False,-1,-1,NaN,3,-1,-1,0,2,0.269677
213,323,False,-1,-1,NaN,3,-1,-1,1,0,0.252283
214,323,False,-1,-1,NaN,3,-1,-1,1,1,0.279924
